In [14]:
!pip install optuna

In [15]:
import os
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

In [16]:
DEVICE = torch.device('cpu')
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()

In [17]:
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_EVAL_EXAMPLES =BATCHSIZE * 10

In [18]:
def defining_model(trial):
  n_layers = trial.suggest_int('n_layers', 1,3)
  layers = []
  in_features = 28*28
  for i in range(n_layers):
    out_features = trial.suggest_int('n_units_l{}'.format(i),4,128)
    layers.append(nn.Linear(in_features,out_features))
    layers.append(nn.ReLU())
    p = trial.suggest_float('dropout_l{}'.format(i),0.2,0.5)
    layers.append(nn.Dropout(p))
    in_features = out_features

  layers.append(nn.Linear(in_features,CLASSES))
  layers.append(nn.LogSoftmax(dim=1))

  return nn.Sequential(*layers)

In [19]:
from random import shuffle
def get_mnist():
  train_loader = torch.utils.data.DataLoader(
      datasets.FashionMNIST(DIR,train=True,download=True,transform=transforms.ToTensor()),
      batch_size = BATCHSIZE,
      shuffle=True
  )

  eval_loader = torch.utils.data.DataLoader(
      datasets.FashionMNIST(DIR,train=False,download=True,transform=transforms.ToTensor()),
      batch_size = BATCHSIZE,
      shuffle=True
  )

  return train_loader,eval_loader

In [20]:
def objective(trial):
  model = defining_model(trial).to(DEVICE)

  optimizer_name = trial.suggest_categorical('optimizer',['Adam','RMSprop','SGD'])
  lr = trial.suggest_float('lr',1e-5,1e-1,log=True)
  optimizer = getattr(optim,optimizer_name)(model.parameters(),lr=lr)

  train_loader,valid_loader = get_mnist()

  for epoch in range(EPOCHS):
    model.train()
    for batch_id,(data,target) in enumerate(train_loader):
      if batch_id*BATCHSIZE >= N_TRAIN_EXAMPLES:
        break

      data,target = data.view(data.size(0),-1).to(DEVICE),target.to(DEVICE)

      optimizer.zero_grad()
      output=model(data)
      loss=F.nll_loss(output,target)
      loss.backward()
      optimizer.step()

    model.eval()
    correct=0
    with torch.no_grad():
      for batch_id,(data,target) in enumerate(valid_loader):
        if batch_id*BATCHSIZE >= N_EVAL_EXAMPLES:
          break
        data,target = data.view(data.size(0),-1).to(DEVICE),target.to(DEVICE)
        output=model(data)
        pred=output.argmax(dim=1,keepdim=True)
        correct+=pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / min(len(valid_loader.dataset),N_EVAL_EXAMPLES)

    trial.report(accuracy,epoch)

    if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

  return accuracy

In [21]:
from traitlets.config import deepcopy
from optuna.exceptions import TrialPruned
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100,timeout=600)

pruned_trials = study.get_trials(deepcopy=False,states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False,states=[TrialState.COMPLETE])

print("States:")
print('Number of finished trials: ',len(study.trials))
print('Number of pruned trials: ',len(pruned_trials))
print('Number of completed trials: ',len(complete_trials))

print('Best trial: ')
trial = study.best_trial
print('Value: ',trial.value)

print('Params: ')
for key,value in trial.params.items():
  print(' {}: {}'.format(key,value))

[I 2023-07-15 22:17:01,262] A new study created in memory with name: no-name-9fd692af-44d3-4804-a5c3-d59906995144
[I 2023-07-15 22:17:14,435] Trial 0 finished with value: 0.69765625 and parameters: {'n_layers': 1, 'n_units_l0': 62, 'dropout_l0': 0.331244223496697, 'optimizer': 'RMSprop', 'lr': 0.02789266713198426}. Best is trial 0 with value: 0.69765625.
[I 2023-07-15 22:17:22,442] Trial 1 finished with value: 0.29375 and parameters: {'n_layers': 2, 'n_units_l0': 89, 'dropout_l0': 0.25328485625039776, 'n_units_l1': 51, 'dropout_l1': 0.26421383494670536, 'optimizer': 'Adam', 'lr': 0.09221156127626035}. Best is trial 0 with value: 0.69765625.
[I 2023-07-15 22:17:31,430] Trial 2 finished with value: 0.71953125 and parameters: {'n_layers': 3, 'n_units_l0': 58, 'dropout_l0': 0.21178493042116792, 'n_units_l1': 111, 'dropout_l1': 0.2915055125070459, 'n_units_l2': 25, 'dropout_l2': 0.2847110344208915, 'optimizer': 'Adam', 'lr': 0.0005859466783321401}. Best is trial 2 with value: 0.71953125.
[I

States:
Number of finished trials:  100
Number of pruned trials:  54
Number of completed trials:  46
Best trial: 
Value:  0.85234375
Params: 
 n_layers: 1
 n_units_l0: 114
 dropout_l0: 0.237830275839296
 optimizer: Adam
 lr: 0.005404841358921655
